# Overfit on one sample
+ Step 1: Uncomment `dataset = dataset.take(1)` in `segmentation/datasets/dataset_utils.py`.
+ Step 2: Modify dataset config.
+ Step 3: Run the following codes.

In [1]:
import sys
sys.path.append('..')
import os
import argparse
import tensorflow as tf
from segmentation.builders import model_builder, dataset_builder
from segmentation.utils import losses_utils, metrics_utils
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/home/ubuntu/anaconda3/envs/zyy_tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/zyy_tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/zyy_tf1.14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/zyy_tf

In [2]:
def parse_args():
    parser = argparse.ArgumentParser()

    # multi-gpu configs
    parser.add_argument('--num_gpus', type=int, default=1)
    parser.add_argument('--gpu_devices', type=str, default="3")

    # dataset
    parser.add_argument('--dataset_name', type=str, default="pascal_voc_seg", help='')
    parser.add_argument('--dataset_dir', type=str, default="/hdd02/zhangyiyang/data/VOCdevkit/segmentation_aug_tfrecords", help='')
    parser.add_argument('--split_name', type=str, default="val", help='')
    parser.add_argument('--eval_crop_height', type=int, default=513)
    parser.add_argument('--eval_crop_width', type=int, default=513)
    parser.add_argument('--max_resize_value', type=int, default=None)
    parser.add_argument('--min_resize_value', type=int, default=None)

    # model related
    parser.add_argument('--model_type', type=str, default="deeplab_v3_plus")
    parser.add_argument('--backend_type', type=str, default="xception")
    parser.add_argument('--model_weights', type=str, default="pascal_voc")
    parser.add_argument('--output_stride', type=int, default=16)
    parser.add_argument('--fine_tune_batch_norm', action='store_true',
                        help='Whether to fine tune bach norm.')

    return parser.parse_args([])
args = parse_args()

In [3]:
def _get_datasets(args):
    val_dataset_configs = dataset_builder.build_dataset_configs(
        dataset_dir=args.dataset_dir,
        batch_size=1,
        crop_size=(args.eval_crop_height, args.eval_crop_width),
        max_resize_value=args.max_resize_value,
        min_resize_value=args.min_resize_value,
        should_shuffle=False,
        is_training=False,
        should_repeat=True,
    )
    val_dataset = dataset_builder.build_dataset(
        args.dataset_name, args.split_name, True, val_dataset_configs)

    return val_dataset

In [4]:
if tf.__version__.split('.')[0] == "1":
    tf.logging.set_verbosity(tf.logging.INFO)
    tf_config = tf.ConfigProto()
    tf_config.gpu_options.allow_growth = True
    sess = tf.Session(config=tf_config)
    tf.keras.backend.set_session(sess)

dataset_meta = dataset_builder.build_dataset_meta(args.dataset_name)
val_dataset = _get_datasets(args)

In [5]:
keras_model = model_builder.build_model(
    model_type=args.model_type,
    backend_type=args.backend_type,
    weights=args.model_weights,
    num_classes=dataset_meta.num_classes,
    OS=args.output_stride,
    input_shape=(args.eval_crop_height, args.eval_crop_width, 3),
    fine_tune_batch_norm=args.fine_tune_batch_norm,
)
keras_model.compile(
    optimizer=tf.keras.optimizers.RMSprop(1e-4),
    loss=losses_utils.build_cross_entropy_loss_fn(dataset_meta.num_classes),
    metrics=[
        metrics_utils.build_accuracy_fn(dataset_meta.num_classes),
        metrics_utils.build_mean_iou_fn(dataset_meta.num_classes)
    ],
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [6]:
keras_model.fit(val_dataset,
               steps_per_epoch=1,
               epochs=30,)

Epoch 1/30
1/1 [==============================] - 14s 14s/step - loss: 0.1588 - _accuracy: 0.9611 - _mean_iou: 0.3487
Epoch 2/30
1/1 [==============================] - 0s 197ms/step - loss: 0.0510 - _accuracy: 0.9832 - _mean_iou: 0.9455
Epoch 3/30
1/1 [==============================] - 0s 202ms/step - loss: 0.0797 - _accuracy: 0.9765 - _mean_iou: 0.6134
Epoch 4/30
1/1 [==============================] - 0s 200ms/step - loss: 0.0461 - _accuracy: 0.9755 - _mean_iou: 0.9243
Epoch 5/30
1/1 [==============================] - 0s 194ms/step - loss: 0.0216 - _accuracy: 0.9912 - _mean_iou: 0.9699
Epoch 6/30
1/1 [==============================] - 0s 196ms/step - loss: 0.0103 - _accuracy: 0.9982 - _mean_iou: 0.9937
Epoch 7/30
1/1 [==============================] - 0s 214ms/step - loss: 0.0076 - _accuracy: 0.9987 - _mean_iou: 0.9957
Epoch 8/30
1/1 [==============================] - 0s 213ms/step - loss: 0.0062 - _accuracy: 0.9991 - _mean_iou: 0.9970
Epoch 9/30
1/1 [==============================] -